In [1]:
import numpy as np
import pandas as pd
import urllib
from bs4 import BeautifulSoup
import urlparse
import mechanize
import requests
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

In [2]:
df = pd.read_csv('Data/basic_info.csv')
# df=df.dropna(axis=0, how='any')
df['原始基金'] = df['原始基金'].replace('[\¥,]', '', regex=True).astype(float)
df

,基金会名称,宗旨,业务范围,成立时间,业务主管单位,票据类型,原始基金,登记部门,注册地,所在地,...,基金会行业领域,全职员工,志愿者数量,专项基金数量,负责人数,负责人中现任国家工作人员数,负责人中担任过省部级及以上领导职务的人数,政府年检,评估等级,优惠资格类型
0,爱佑慈善基金会,接受并管理捐赠资金，开展社会救助，促进慈善事业发展。,（一）设立和资助孤贫、残疾儿童的助养、医疗救助项目；\r（二）开展和资助有利于少年儿童健康成...,2008-05-06,民政部,捐赠票据,20000000.0,民政部,民政部,北京市朝阳区,...,儿童，医疗救助,39.0,221.0,0.0,4.0,0.0,0.0,合格,5A,"公益性捐赠税前扣除资格, 非营利组织免税资格"
1,北京大学教育基金会,遵守中华人民共和国宪法、法律、法规和国家政策，遵守社会道德规范，致力于加强北京大学与国内外各...,接受和管理社会各界的捐赠实现基金的保值、增值 奖励教师、学生，资助教学，科研等。,1995-07-04,教育部,"捐赠票据, 其他",20000000.0,民政部,民政部,北京市海淀区,...,教育，科学研究,30.0,19.0,0.0,3.0,0.0,1.0,合格,5A,公益性捐赠税前扣除资格
2,北京交通大学教育基金会,通过资金筹集及进行资助，推动教育事业的发展。,奖教奖学 资助贫困 课题研究 学术研讨 专业培训 国际合作 咨询服务,2009-07-29,教育部,"捐赠票据, 其他",20000000.0,民政部,民政部,北京市海淀区,...,教育，科学研究,10.0,33.0,0.0,5.0,0.0,0.0,合格,4A,"公益性捐赠税前扣除资格, 非营利组织免税资格"
3,传媒大学教育基金会,争取国内外团体与个人的支持与捐助，加强中国传媒大学与国内外各界的联系与合作，推动中国传媒大学...,专项资助、捐资助学、咨询培训。,2007-06-21,教育部,捐赠票据,20000000.0,民政部,民政部,北京市朝阳区,...,国际事务，教育，科学研究,7.0,30.0,0.0,3.0,0.0,0.0,合格,1A,"公益性捐赠税前扣除资格, 非营利组织免税资格"
4,海仓慈善基金会,扶贫助残、促进环保、努力构建和谐社会。重点资助城乡孤寡老人、残疾人及教育事业。,资助社会福利事业、扶贫济困、赈灾救援、助学兴教和慈善宣传活动。资助和奖励促进职工教育、环境保...,2008-12-18,民政部,捐赠票据,50000000.0,民政部,民政部,北京市东城区,...,安全救灾，扶贫助困，教育,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,华民慈善基金会,本基金会致力于在市场经济条件下，秉承诚信、专业、规范、透明和高效的原则，探索发展中国特色现代...,1、资助老年机构建设及老年福利服务；\r\r2、大学生就业扶助；\r\r3、资助孤寡病残等民...,2008-03-11,民政部,捐赠票据,50000000.0,民政部,民政部,北京市东城区,...,安全救灾，创业就业，扶贫助困，老年人,15.0,25.0,0.0,2.0,0.0,0.0,合格,4A,公益性捐赠税前扣除资格
6,华润慈善基金会,常怀感恩之心，不忘回馈大众，构建和谐社会\r,扶贫及社会主义新农村建设，赈灾济难，扶危济困，困难群体生活保障，促进教育，环境保护,2010-01-14,民政部,捐赠票据,50000000.0,民政部,民政部,广东省深圳市罗湖区,...,安全救灾，扶贫助困，环境，三农,1.0,140.0,0.0,3.0,0.0,0.0,合格,5A,公益性捐赠税前扣除资格
7,凯风公益基金会,推动社会进步，促进和谐发展,开展公益慈善活动，资助社会保障、社会福利等方面的课题,2007-03-01,民政部,捐赠票据,50000000.0,民政部,民政部,北京市朝阳区,...,扶贫助困，公益事业发展,5.0,0.0,0.0,2.0,0.0,0.0,合格,3A,非营利组织免税资格
8,李四光地质科学奖基金会,纪念李四光，弘扬其科学求实创新和爱国主义精神，推动我国地质工作可持续发展。,学术交流、专项资助、国际合作、咨询服务,2007-04-03,国土资源部,"捐赠票据, 税务发票",20000000.0,民政部,民政部,北京市西城区,...,科学研究，文化,0.0,0.0,0.0,4.0,1.0,0.0,合格,2A,"公益性捐赠税前扣除资格, 非营利组织免税资格"
9,南都公益基金会,"关注转型期的中国社会问题，资助优秀公益项目,推动民间组织的社会创新，促进社会平等和谐；使社会...",专项资助、教育培训、国际合作、咨询服务。,2007-05-11,民政部,捐赠票据,100000000.0,民政部,民政部,北京市朝阳区,...,创业就业，公益事业发展，教育,23.0,17.0,0.0,2.0,0.0,0.0,合格,4A,"公益性捐赠税前扣除资格, 非营利组织免税资格"


In [3]:
fina_df = pd.read_csv('Data/fina_info.csv')
fina_df

,基金会名称,财务年度,净资产,总收入,捐赠收入,投资收益,服务收入,政府补助收入,其他收入,业务活动成本,...,费用合计,总支出,公益事业支出,工资福利支出,行政办公支出,其他支出,法人捐赠,自然人捐赠,境内捐赠,境外捐赠
0,北海天宁慈善基金会,2013,1.918713e+06,3.405616e+04,2.156000e+04,12496.16,0.00,0.00,0.00,2.500000e+05,...,2.225375e+05,3.874860e+05,2.582000e+05,30000.00,99286.00,0.00,2.000000e+04,1560.00,2.156000e+04,0.000000e+00
1,福建农林大学教育发展基金会,2013,1.686086e+07,1.090801e+07,1.069428e+07,0.00,0.00,0.00,213726.50,1.400784e+06,...,1.446926e+06,1.446926e+06,1.400784e+06,39700.00,6442.40,NaN,1.059428e+07,100000.00,1.045000e+07,2.442840e+05
2,福建省扶贫基金会,2013,1.049211e+07,6.425691e+06,5.375000e+06,446660.00,0.00,600000.00,4031.00,6.040397e+06,...,6.273203e+06,6.037706e+06,5.800000e+06,118485.12,77320.85,41900.00,0.000000e+00,5375000.00,5.375000e+06,0.000000e+00
3,广西妇女儿童发展基金会,2013,6.614637e+06,3.898046e+06,3.389580e+06,0.00,0.00,0.00,508465.64,2.591636e+06,...,2.772101e+06,2.772101e+06,2.591636e+06,59580.70,12814.07,108070.11,3.319580e+06,70000.00,3.389580e+06,0.000000e+00
4,广西国海扶贫助学基金会,2013,3.040783e+06,1.440190e+03,0.000000e+00,0.00,0.00,0.00,1440.19,3.039000e+05,...,7.771187e+04,3.039000e+05,3.039000e+05,0.00,0.00,0.00,0.000000e+00,0.00,0.000000e+00,0.000000e+00
5,宁夏回族自治区关心下一代基金会,2013,4.830445e+06,8.730573e+05,7.096660e+05,0.00,0.00,0.00,163391.31,7.322463e+05,...,7.841226e+05,7.841226e+05,7.322463e+05,42632.01,9244.23,0.00,6.284095e+05,81256.48,7.096660e+05,0.000000e+00
6,福建省龙岩扶贫基金会,2013,9.574226e+06,9.013735e+05,2.000000e+05,0.00,0.00,0.00,701373.49,8.800000e+05,...,9.509380e+05,9.509380e+05,8.800000e+05,41130.00,29808.00,0.00,2.000000e+05,0.00,2.000000e+05,0.000000e+00
7,福建省石竹慈善基金会,2013,3.702224e+06,4.741631e+06,4.724100e+06,0.00,0.00,0.00,17531.42,1.105300e+07,...,1.122363e+07,9.053000e+06,9.053000e+06,0.00,0.00,0.00,4.724100e+06,0.00,4.724100e+06,0.000000e+00
8,广西李宁基金会,2013,5.068391e+06,4.003042e+06,3.996708e+06,0.00,0.00,0.00,6334.11,2.422148e+06,...,3.275126e+06,3.275126e+06,2.422148e+06,337268.51,515709.38,0.00,3.996708e+06,0.00,8.999900e+04,3.906709e+06
9,广西吕贻标爱心基金会,2013,1.012347e+07,1.054200e+07,1.003000e+07,512000.00,0.00,0.00,0.00,2.571870e+05,...,2.813916e+05,4.185271e+05,2.285150e+05,68911.73,14389.00,106711.35,1.003000e+07,0.00,1.003000e+07,0.000000e+00


In [147]:
invalid_organizations = fina_df.loc[fina_df['工资福利支出'] / fina_df['总支出'] > 0.1,'基金会名称']
invalid_organizations

8                  广西李宁基金会
9               广西吕贻标爱心基金会
34            湖南省老龄事业发展基金会
52              海门文明村爱心基金会
55               中国发展研究基金会
104               盐城市扶贫基金会
118           贵州省青年创业就业基金会
153             杭州市关爱孤儿基金会
184           陕西省老龄事业发展基金会
216              重庆光彩事业基金会
252        新疆维吾尔自治区新勇教育基金会
302              吴江盛虹爱心基金会
315           青海省雪域仁济慈善基金会
332             广东省环境保护基金会
357             梅州市希望教育基金会
374               中国艺术节基金会
520            北京友好传承文化基金会
528              北京彩虹公益基金会
560                广西李宁基金会
607             广东省鸿发慈善基金会
687              吴江盛虹爱心基金会
764             广东省卓越慈善基金会
796             北京市光明慈善基金会
809              泉州向善救助基金会
854            福建海西青年创业基金会
876             北京新阳光慈善基金会
897               中国艺术节基金会
938             武汉善之源公益基金会
950            云南省关心下一代基金会
960            重庆宣传文化发展基金会
965           广东省烛光教育发展基金会
974             中国交响乐发展基金会
978               盐城市扶贫基金会
984          贵州文化薪火乡村发展基金会
989               广州市禁毒基金会
1033            杭州市关爱孤儿基金会
1055       新疆维吾尔自治区新勇教育基金会
1

In [4]:
invest_df = pd.read_csv('Data/invest_info.csv')
invest_df
# len(invest_df['受助单位名称'].unique())

,项目编码,受助单位名称,资助金额
0,63990,爱佑慈善基金会,3000000.00
1,64114,爱佑慈善基金会,3500000.00
2,65610,爱佑慈善基金会,1090000.00
3,70032,爱佑慈善基金会,2000000.00
4,80714,爱佑慈善基金会,3500000.00
5,80643,爱佑慈善基金会,2081221.77
6,83721,爱佑慈善基金会,2538045.69
7,62873,爱佑慈善基金会,4700000.00
8,90092,爱佑慈善基金会,500000.00
9,91195,爱佑慈善基金会,5000000.00


In [5]:
project_df = pd.read_csv('Data/project.csv')
project_df = project_df.loc[project_df['项目编码'].str.isnumeric(),:]
project_df['项目编码'] = project_df['项目编码'].astype(np.int32)
project_df


,基金会名称,项目年度,项目编码,项目名称,项目收入,项目支出,项目详情,关注领域,覆盖地域,SDG
0,深圳壹基金公益基金会,2013,18065,公益人才培养项目,1003700,2429589.12,项目1：公益视频征集和评选大赛。壹基金公益映像节自7月8日启动起来，经过四个月的征集、投票和...,公益行业发展,北京市海淀区，广东省深圳市,优质教育
1,厦门观音寺慈善基金会,2013,21059,医疗-2012,22626,58626,其中捐张宜白血病治疗费0.6万元.捐张彩娟医药费2.2626万元.捐陈忠生医药费3万元.,医疗救助,福建省厦门市,其他
2,浙江敦和慈善基金会,2013,23855,成都城市河流研究会长江上游清流计划,0,100000,浙江敦和慈善基金会捐赠成都城市河流研究会10万元，用于长江上游清流计划——川西乡村行动项目。,公益行业发展,天津市宁河区，天津市宁河区,可持续城市和社区
3,青岛市天泰公益基金会,2013,26100,捐助“青岛即墨爱之家弱智儿童救助站”项目。,0,15357,位于即墨段泊岚镇瓦戈庄的青岛爱之家弱智儿童寄托站是天泰基金会定点捐助的项目。2012年，该寄...,扶贫助困,山东省青岛市即墨市,优质教育
4,福建农林大学教育发展基金会,2013,61112,软件学院2013届毕业生双困助学金,600000,120000,资助学生,教育,福建省福州市仓山区,可持续城市和社区
5,福建农林大学教育发展基金会,2013,61117,2013届双困毕业生就业援助金,600000,206200,资助学生,教育,福建省,清洁饮水和卫生设施
6,福建农林大学教育发展基金会,2013,61120,2012年里昂助学金,187583.9,187583.9,资助学生,教育,福建省,可持续城市和社区
7,福建农林大学教育发展基金会,2013,61123,庄卫汉奖学金,180000,170000,资助学生,教育,福建省,其他
8,福建农林大学教育发展基金会,2013,61125,2012年圣农助学金,20000,20000,资助学生,教育,福建省,可持续城市和社区
9,北海天宁慈善基金会,2013,61234,捐助希望高中,0,60000,给广西青少年发展基金会每年捐助6万元做希望高中助学,教育,广西壮族自治区北海市,优质教育


In [128]:
transparency_df = pd.read_csv('Data/transparency_info.csv').rename(columns={'分数':'score','基金会名称':'基金会名称','排名':'排名'})
transparency_df.loc[:,'score'].fillna(transparency_df.loc[~transparency_df['score'].isnull()].mean(),inplace=True)
transparency_df['score'] -= transparency_df['score'].min()
transparency_df['score'] /= transparency_df['score'].max()
print transparency_df['score'].max(), transparency_df['score'].min()

1.0 0.0


In [113]:
# collect all organizations
all_organizations = list(set(list(set(invest_df.loc[:,'受助单位名称'].unique())) + list(set(project_df.loc[:,'基金会名称'])) +list(set(df.loc[:,'基金会名称']))))

In [8]:
fieldSet = set()
def create_field_set(row):
    global fieldSet
    fieldList = row.split('，')
    for field in fieldList:
        if field not in fieldSet:
            fieldSet.add(field)

In [9]:
#create field set for encoding
fieldDf = df.loc[:,'基金会行业领域']
fieldDf.apply(create_field_set)
fieldSet

{'\xe4\xb8\x89\xe5\x86\x9c',
 '\xe4\xbd\x93\xe8\x82\xb2',
 '\xe4\xbe\xa8\xe5\x8a\xa1',
 '\xe5\x84\xbf\xe7\xab\xa5',
 '\xe5\x85\xac\xe5\x85\xb1\xe5\xae\x89\xe5\x85\xa8',
 '\xe5\x85\xac\xe5\x85\xb1\xe6\x9c\x8d\xe5\x8a\xa1',
 '\xe5\x85\xac\xe7\x9b\x8a\xe4\xba\x8b\xe4\xb8\x9a\xe5\x8f\x91\xe5\xb1\x95',
 '\xe5\x88\x9b\xe4\xb8\x9a\xe5\xb0\xb1\xe4\xb8\x9a',
 '\xe5\x8a\xa8\xe7\x89\xa9\xe4\xbf\x9d\xe6\x8a\xa4',
 '\xe5\x8c\xbb\xe7\x96\x97\xe6\x95\x91\xe5\x8a\xa9',
 '\xe5\x8d\xab\xe7\x94\x9f\xe4\xbf\x9d\xe5\x81\xa5',
 '\xe5\x9b\xbd\xe9\x99\x85\xe4\xba\x8b\xe5\x8a\xa1',
 '\xe5\xa6\x87\xe5\xa5\xb3',
 '\xe5\xae\x89\xe5\x85\xa8\xe6\x95\x91\xe7\x81\xbe',
 '\xe5\xb0\x91\xe6\x95\xb0\xe6\xb0\x91\xe6\x97\x8f',
 '\xe5\xbf\x83\xe7\x90\x86\xe5\x81\xa5\xe5\xba\xb7',
 '\xe5\xbf\x97\xe6\x84\xbf\xe6\x9c\x8d\xe5\x8a\xa1',
 '\xe6\x89\xb6\xe8\xb4\xab\xe5\x8a\xa9\xe5\x9b\xb0',
 '\xe6\x95\x99\xe8\x82\xb2',
 '\xe6\x96\x87\xe5\x8c\x96',
 '\xe6\xae\x8b\xe7\x96\xbe',
 '\xe6\xb3\x95\xe5\xbe\x8b\xe5\xae\x9e\xe6\x96\xbd',
 '

In [10]:
#transform field information into multi-hot encoding
fieldList = list(fieldSet)
def multihot_encoding(row):
    global fieldSet
    global fieldList
    field_encoding = np.zeros(len(fieldSet))
    fields = row.split('，')
    for field in fields:
        if field in fieldSet:
            index = fieldList.index(field)
            field_encoding[index] = 1
    return field_encoding

In [11]:
multihotField = fieldDf.apply(multihot_encoding)
multihotField

0      [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...
1      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4      [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
5      [1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...
6      [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
7      [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
8      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
9      [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...
10     [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...
11     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
12     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...
13     [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
14     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...
15     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
16     [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
17     [1.0, 0.0, 0.0, 0.0, 1.0

In [12]:
# calculate the 
fieldSimilarityDf = pd.DataFrame(columns = df['基金会名称'], index=df['基金会名称'])
length,_ = fieldSimilarityDf.shape
# compute the cosine similarity
fieldSimilarityDf = pd.DataFrame(cosine_similarity(np.array(list(multihotField))), columns = df['基金会名称'], index=df['基金会名称'])
fieldSimilarityDf

基金会名称,爱佑慈善基金会,北京大学教育基金会,北京交通大学教育基金会,传媒大学教育基金会,海仓慈善基金会,华民慈善基金会,华润慈善基金会,凯风公益基金会,李四光地质科学奖基金会,南都公益基金会,...,宁波市善园公益基金会,中天爱心慈善基金会,武汉科技大学教育发展基金会,北京白求恩公益基金会,北京市银杏公益基金会,北京麦田慈善基金会,内蒙古光彩事业基金会,晋江市安海中心小学种胜教育基金会,东润公益基金会,广东省广铝公益基金会
基金会名称,,,,,,,,,,,,,,,,,,,,,
爱佑慈善基金会,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
北京大学教育基金会,0.000000,1.000000,1.000000,0.816497,0.408248,0.000000,0.000000,0.000000,0.500000,0.408248,...,0.0,0.000000,0.707107,0.000000,0.707107,0.000000,0.500000,0.707107,0.408248,0.408248
北京交通大学教育基金会,0.000000,1.000000,1.000000,0.816497,0.408248,0.000000,0.000000,0.000000,0.500000,0.408248,...,0.0,0.000000,0.707107,0.000000,0.707107,0.000000,0.500000,0.707107,0.408248,0.408248
传媒大学教育基金会,0.000000,0.816497,0.816497,1.000000,0.333333,0.000000,0.000000,0.000000,0.408248,0.333333,...,0.0,0.000000,0.577350,0.000000,0.577350,0.000000,0.408248,0.577350,0.333333,0.333333
海仓慈善基金会,0.000000,0.408248,0.408248,0.333333,1.000000,0.577350,0.577350,0.408248,0.000000,0.333333,...,0.0,0.000000,0.577350,0.000000,0.000000,0.408248,0.816497,0.577350,1.000000,0.666667
华民慈善基金会,0.000000,0.000000,0.000000,0.000000,0.577350,1.000000,0.500000,0.353553,0.000000,0.288675,...,0.0,0.000000,0.000000,0.000000,0.000000,0.353553,0.353553,0.000000,0.577350,0.577350
华润慈善基金会,0.000000,0.000000,0.000000,0.000000,0.577350,0.500000,1.000000,0.353553,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.707107,0.353553,0.000000,0.577350,0.288675
凯风公益基金会,0.000000,0.000000,0.000000,0.000000,0.408248,0.353553,0.353553,1.000000,0.000000,0.408248,...,0.0,0.707107,0.000000,0.000000,0.000000,0.500000,0.500000,0.000000,0.408248,0.408248
李四光地质科学奖基金会,0.000000,0.500000,0.500000,0.408248,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.707107,0.000000,0.000000,0.000000,0.000000,0.000000


In [13]:
bi_invest_df = invest_df.loc[:,['受助单位名称','项目编码']].join(project_df.loc[:,['基金会名称','项目编码','SDG']].set_index('项目编码'), on='项目编码').dropna(subset=['基金会名称'])
bi_invest_df

,受助单位名称,项目编码,基金会名称,SDG
0,爱佑慈善基金会,63990,腾讯公益慈善基金会,良好健康与福祉，无贫穷
1,爱佑慈善基金会,64114,招商局慈善基金会,良好健康与福祉
2,爱佑慈善基金会,65610,北京百度公益基金会,良好健康与福祉，无贫穷
3,爱佑慈善基金会,70032,中华思源工程扶贫基金会,无贫穷，优质教育
4,爱佑慈善基金会,80714,万科公益基金会,良好健康与福祉，性别平等
5,爱佑慈善基金会,80643,万科公益基金会,良好健康与福祉，无贫穷
6,爱佑慈善基金会,83721,中国儿童少年基金会,良好健康与福祉，零饥饿，无贫穷
7,爱佑慈善基金会,62873,阿里巴巴公益基金会,良好健康与福祉
9,爱佑慈善基金会,91195,顺丰公益基金会,良好健康与福祉，无贫穷
10,爱佑慈善基金会,91211,万科公益基金会,良好健康与福祉


In [14]:
# record the investment history

investedDf = pd.DataFrame(columns = all_organizations, index=all_organizations).fillna(0)
investedDf

,杭州市生态文化协会,帝斯曼维生素（上海）有限公司,浙江省人民医院,北京市华远集团公司,北京富润德斯文化发展有限公司,北京修实公益基金会,北京世纪银通投资有限公司,北京欧亚美展览服务有限公司,中国科学院上海微系统与信息技术研究所,云南商学院,...,中国少先队事业发展中心,北京中科青云实验学校,昆山育才学校,北京市海淀外国语实验学校,政和县教育局,河南鸿翔一心堂药业有限公司郑州金水路店,陕西巾帼家庭教育网络信息服务中心,山东省青年创业就业基金会,宁夏志愿者协会,青海省果洛州久治县哇尔依乡藏文寄宿制小学
杭州市生态文化协会,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
帝斯曼维生素（上海）有限公司,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
浙江省人民医院,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
北京市华远集团公司,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
北京富润德斯文化发展有限公司,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
北京修实公益基金会,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
北京世纪银通投资有限公司,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
北京欧亚美展览服务有限公司,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
中国科学院上海微系统与信息技术研究所,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
云南商学院,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
def count_invest(df):
    for index, row in bi_invest_df.iterrows():
        helped = row['受助单位名称']
        helping = row['基金会名称']
        if helped not in df.index or  helping not in df.index:
            print help, helping
            
            continue
        
        df.loc[helping, helped] +=1  
    return df
investedDf = count_invest(investedDf)

In [16]:
SDGList = ['无贫穷',
'零饥饿',
'良好健康与福祉',
'优质教育',
'性别平等',
'清洁饮水和卫生设施',
'经济适用的清洁能源',
'体面工作和经济增长',
'产业、创新和基础设施',
'减少不平等',
'可持续城市和社区',
'负责任消费和生产',
'气候行动',
'水下生物',
'陆地生物',
'和平、正义与强大机构',
'促进目标实现的伙伴关系',
'其他']
SDGSet = set(SDGList)
execute_organizations = bi_invest_df['受助单位名称'].unique()
executeDict = dict()

for e_organization in execute_organizations:
    currentDf = bi_invest_df.loc[bi_invest_df['受助单位名称'] == e_organization,['受助单位名称','SDG']]
    SDGs = []
    for index, row in currentDf.iterrows():
        SDGs += row['SDG'].split('，')  
    executeDict[currentDf['受助单位名称'].iloc[0]] = SDGs
SDGDf = pd.DataFrame(executeDict.items()).rename(columns={0:'受助单位名称',1:'SDG'})


In [17]:
def SDGMultiHotEncoding(row):
    global SDGList
    currentEncoding = np.zeros(len(SDGList))
    for SDG in row:
        index = SDGList.index(SDG)
        currentEncoding[index] = 1
    return currentEncoding

In [18]:
SDGDf['SDG'] = SDGDf['SDG'].apply(SDGMultiHotEncoding)
SDGDf

,受助单位名称,SDG
0,杭州市生态文化协会,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,帝斯曼维生素（上海）有限公司,"[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,左云县林业局,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,北京市华远集团公司,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,北京富润德斯文化发展有限公司,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,北京修实公益基金会,"[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,北京世纪银通投资有限公司,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7,中国科学院上海微系统与信息技术研究所,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,云南商学院,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,米易县麻陇乡财政所,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [19]:
SDGSimilarityDf = pd.DataFrame(cosine_similarity(np.array(list(SDGDf['SDG']))), columns=SDGDf['受助单位名称'], index= SDGDf['受助单位名称'])
SDGSimilarityDf

受助单位名称,杭州市生态文化协会,帝斯曼维生素（上海）有限公司,左云县林业局,北京市华远集团公司,北京富润德斯文化发展有限公司,北京修实公益基金会,北京世纪银通投资有限公司,中国科学院上海微系统与信息技术研究所,云南商学院,米易县麻陇乡财政所,...,汶川县教育局,中国少先队事业发展中心,北京中科青云实验学校,昆山育才学校,北京欧亚美艺术展览服务有限公司,政和县教育局,河南鸿翔一心堂药业有限公司郑州金水路店,陕西巾帼家庭教育网络信息服务中心,宁夏志愿者协会,青海省果洛州久治县哇尔依乡藏文寄宿制小学
受助单位名称,,,,,,,,,,,,,,,,,,,,,
杭州市生态文化协会,1.000000,0.000000,0.000000,1.000000,1.000000,0.707107,0.000000,1.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.707107,1.000000,0.707107,0.000000,0.000000,0.000000,0.000000
帝斯曼维生素（上海）有限公司,0.000000,1.000000,0.000000,0.000000,0.000000,0.408248,0.000000,0.000000,0.577350,0.000000,...,0.000000,0.000000,0.000000,0.408248,0.000000,0.408248,0.577350,0.000000,0.000000,0.577350
左云县林业局,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.707107,0.000000,0.000000
北京市华远集团公司,1.000000,0.000000,0.000000,1.000000,1.000000,0.707107,0.000000,1.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.707107,1.000000,0.707107,0.000000,0.000000,0.000000,0.000000
北京富润德斯文化发展有限公司,1.000000,0.000000,0.000000,1.000000,1.000000,0.707107,0.000000,1.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.707107,1.000000,0.707107,0.000000,0.000000,0.000000,0.000000
北京修实公益基金会,0.707107,0.408248,0.000000,0.707107,0.707107,1.000000,0.000000,0.707107,0.707107,0.000000,...,0.000000,0.707107,0.000000,1.000000,0.707107,1.000000,0.000000,0.000000,0.000000,0.707107
北京世纪银通投资有限公司,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
中国科学院上海微系统与信息技术研究所,1.000000,0.000000,0.000000,1.000000,1.000000,0.707107,0.000000,1.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.707107,1.000000,0.707107,0.000000,0.000000,0.000000,0.000000
云南商学院,0.000000,0.577350,0.000000,0.000000,0.000000,0.707107,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.707107,0.000000,0.707107,0.000000,0.000000,0.000000,1.000000


In [30]:
# create a numpy array for quick operation
invested2D = np.zeros((len(all_organizations),len(all_organizations)))
for i in range(len(all_organizations)):
    helping = all_organizations[i]
#     print bi_invest_df['基金会名称']
    if helping in set(bi_invest_df['基金会名称']):
        # gather all the organizations that helping has invested
        hashelpeds = list(bi_invest_df.loc[bi_invest_df['基金会名称'] == helping,'受助单位名称'].unique())
    else:
        continue        
        
    # if there is no invested organizations for this helping organization
    if len(hashelpeds) ==0:
        continue
    for j in range(len(all_organizations)):
        
        helped = all_organizations[j]
        if helping in fieldSimilarityDf.index and helped in SDGSimilarityDf.index:         
            if len(hashelpeds) == 0:
                invested2D[i][j] = 0
            else:
                num_hashelped = 0
                accumulatedSimilarity = 0
#                 print helping
#                 print (hashelpeds[0])
#                 print helped
#                 print "---"
                for hashelped in hashelpeds:
            
                    if hashelped in SDGSimilarityDf.index and hashelped in fieldSimilarityDf.index:
#                         print hashelped
                        num_hashelped +=1
                        accumulatedSimilarity += SDGSimilarityDf[hashelped][helped] * fieldSimilarityDf[helping][hashelped]
                if num_hashelped == 0:
                    invested2D[i,j] = 0
                else:
                    invested2D[i,j] = accumulatedSimilarity/num_hashelped
#                     print invested2D[i,j]
        else:
            invested2D[i][j] = 0
investedSimilarity = pd.DataFrame(invested2D,columns=all_organizations, index = all_organizations)
investedSimilarity
# sum(sum(invested2D))



  4%|▍         | 156/3821 [00:21<08:14,  7.42it/s]

,杭州市生态文化协会,帝斯曼维生素（上海）有限公司,浙江省人民医院,北京市华远集团公司,北京富润德斯文化发展有限公司,北京修实公益基金会,北京世纪银通投资有限公司,北京欧亚美展览服务有限公司,中国科学院上海微系统与信息技术研究所,云南商学院,...,中国少先队事业发展中心,北京中科青云实验学校,昆山育才学校,北京市海淀外国语实验学校,政和县教育局,河南鸿翔一心堂药业有限公司郑州金水路店,陕西巾帼家庭教育网络信息服务中心,山东省青年创业就业基金会,宁夏志愿者协会,青海省果洛州久治县哇尔依乡藏文寄宿制小学
杭州市生态文化协会,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
帝斯曼维生素（上海）有限公司,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
浙江省人民医院,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
北京市华远集团公司,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
北京富润德斯文化发展有限公司,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
北京修实公益基金会,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
北京世纪银通投资有限公司,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
北京欧亚美展览服务有限公司,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
中国科学院上海微系统与信息技术研究所,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
云南商学院,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000


<hr>

In [158]:
# ability of execution 

manforceDf = df.loc[:,['基金会名称','全职员工','志愿者数量', '负责人数']]
manforceDf.fillna(0, inplace =True)
manforceDf['man_force'] = manforceDf.apply(lambda(row): row['全职员工'] + row['志愿者数量'] + row['负责人数'], axis = 1)
manforceDf  = manforceDf.loc[:,['基金会名称', 'man_force']].set_index('基金会名称')
manforceDf['man_force'] /= manforceDf['man_force'].max()
print manforceDf['man_force'].max(), manforceDf['man_force'].min()

1.0 0.0


In [157]:
leaderDf = df.loc[:,['基金会名称','负责人中现任国家工作人员数','负责人中担任过省部级及以上领导职务的人数']]
leaderDf.fillna(0, inplace =True)
leaderDf['leader_num'] = leaderDf.apply(lambda(row): row['负责人中现任国家工作人员数'] + row['负责人中担任过省部级及以上领导职务的人数'],axis =1)
leaderDf=leaderDf.loc[:,['基金会名称','leader_num']].set_index('基金会名称')
leaderDf['leader_num'] /= leaderDf['leader_num'].max()
print leaderDf['leader_num'].max(), leaderDf['leader_num'].min()

1.0 0.0


<hr>

In [155]:
# portion of expenditure on social service in 2015
servicePortionDf = fina_df.loc[fina_df['财务年度']==2015,['基金会名称','总支出','公益事业支出']]
servicePortionDf['service_portion'] = servicePortionDf.loc[servicePortionDf['总支出']>0,:].apply(lambda(row): row['公益事业支出'] / row['总支出'] , axis=1)
servicePortionDf['service_portion'].fillna(0, inplace=True)
servicePortionDf = servicePortionDf.loc[:, ['基金会名称', 'service_portion']].set_index('基金会名称')
servicePortiondf = servicePortionDf.loc[servicePortionDf['service_portion']>1,'service_portion'] =1
print servicePortionDf.max(), servicePortionDf.min()

service_portion    1.0
dtype: float64 service_portion    0.0
dtype: float64


In [91]:
# improve rate of total assests & social service expenditure
fina_df201415 = fina_df.loc[(fina_df['财务年度'] == 2015) | (fina_df['财务年度'] == 2014),:]
organizations = list(fina_df201415['基金会名称'].unique())
operationDf = pd.DataFrame(columns = ['total','service'], index = organizations)
fina_df201415 = fina_df201415.loc[fina_df201415['财务年度'] == 2015,['基金会名称','净资产','公益事业支出']].join( fina_df201415.loc[fina_df201415['财务年度'] == 2014,['基金会名称','净资产','公益事业支出']].rename(columns={'基金会名称':'基金会名称','净资产':'净资产2014','公益事业支出':'公益事业支出2014'}).set_index('基金会名称'), on='基金会名称')
total_mean = fina_df201415.loc[(fina_df201415['净资产'] >0) & (fina_df201415['净资产2014']>0),:].apply(lambda(row): (row['净资产'] - row['净资产2014'])/row['净资产2014'],axis=1).mean()
service_mean = fina_df201415.loc[(fina_df201415['公益事业支出'] >0) & (fina_df201415['公益事业支出2014']>0),:].apply(lambda(row): (row['公益事业支出'] - row['公益事业支出2014'])/row['公益事业支出2014'],axis=1).mean()
fina_df201415 = fina_df.loc[(fina_df['财务年度'] == 2015) | (fina_df['财务年度'] == 2014),:]
for organization in organizations:
    
#     print organization
    df2014 = fina_df201415.loc[(fina_df201415['基金会名称'] == organization )&(fina_df201415['财务年度'] == 2014), ['净资产','公益事业支出']]
    df2015 = fina_df201415.loc[(fina_df201415['基金会名称'] == organization )&(fina_df201415['财务年度'] == 2015), ['净资产','公益事业支出']]
#     print (df2015.iloc[0]['净资产']-df2014.iloc[0]['净资产'])
    if df2014.shape[0] <1 or df2015.shape[0] <1:
        operationDf.loc[operationDf.index ==organization,'total']= total_mean
        operationDf.loc[operationDf.index ==organization,'service']=service_mean
    else:
        operationDf.loc[operationDf.index==organization,'total'] = (df2015.iloc[0]['净资产']-df2014.iloc[0]['净资产'])/ df2014.iloc[0]['净资产']
        operationDf.loc[operationDf.index==organization,'service']=(df2015.iloc[0]['公益事业支出']-df2014.iloc[0]['公益事业支出'])/df2014.iloc[0]['公益事业支出']
        # fill the incomplete info with mean value
        if df2014.iloc[0]['净资产'] == 0 or df2015.iloc[0]['净资产'] == 0 : operationDf.loc[operationDf.index==organization,'total'] = total_mean
        if df2014.iloc[0]['公益事业支出'] == 0 or df2015.iloc[0]['公益事业支出'] == 0 : operationDf.loc[operationDf.index==organization,'service'] = service_mean
operationDf



/Users/steevehuang/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in double_scalars
/Users/steevehuang/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in double_scalars
/Users/steevehuang/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in double_scalars


,total,service
神华公益基金会,-0.201371,0.351607
中国癌症基金会,0.641242,0.384563
中国经济改革研究基金会,0.243661,1.3767
中国留学人才发展基金会,-0.346734,-0.547008
山东省普觉公益基金会,0.210568,1.27922
中华少年儿童慈善救助基金会,1.18477,-0.099485
河北省妇女儿童发展基金会,1.4407,0.163988
河南省华夏历史文明传承创新基金会,-0.0100126,2.2082
福建农林大学教育发展基金会,0.0151519,0.219482
福建省发展体育事业基金会,0.189576,0.0684834


In [92]:
# normalize
operationDf -= operationDf.min()
operationDf /= operationDf.max()
operationDf.max()

total      1.0
service    1.0
dtype: float64

In [166]:
# total asset in 2015
totalAssetDf = fina_df.loc[fina_df['财务年度']==2015,['基金会名称','净资产']].set_index('基金会名称')
totalAssetDf['净资产'] = totalAssetDf['净资产']/totalAssetDf['净资产'].max()
totalAssetDf['净资产'].max()

1.0

In [171]:
# total service expenditure in 2015
serviceExpendDf = fina_df.loc[fina_df['财务年度']==2015,['基金会名称','净资产']].set_index('基金会名称').dropna()
serviceExpendDf['净资产'] = serviceExpendDf['净资产']/serviceExpendDf['净资产'].max()
totalAssetDf['净资产'].max()

1.0

<h1>News Scraper</h1>

In [168]:
# results from scraper.py
scraperDf = pd.read_csv('organization_dict10000.csv')
scraperDf.rename(columns={'0':'基金会名称','1':'count'}, inplace=True)
scraperDf = scraperDf.set_index('基金会名称')
scraperDf['count'] = scraperDf['count']/scraperDf['count'].max()
scraperDf.max()

count    1.0
dtype: float64

<h1>Final Score</h1>

In [194]:
INVESTING = '中国红十字基金会'
SIMILARITY_WEIGHT = 40
INFLUENCE_WEIGHT = 30
EXECUTION_WEIGHT = 30
# print all_organizations
all_organizations_except_investing = list(all_organizations)
all_organizations_except_investing.pop(all_organizations.index(INVESTING))
print len(all_organizations_except_investing), len(all_organizations)
score_dict = dict(zip(all_organizations_except_investing, np.zeros(len(all_organizations_except_investing))))
similarities = np.zeros(len(all_organizations_except_investing))
influences = np.zeros(len(all_organizations_except_investing))
executions = np.zeros(len(all_organizations_except_investing))
for i, organization in enumerate(all_organizations_except_investing):
    # similarity
    
    #direct similarity based on field
    if organization in fieldSimilarityDf and INVESTING in fieldSimilarityDf:
        
        similarities[i] += fieldSimilarityDf[organization][INVESTING]
    # invest similarity based on SDGs and invest history
    similarities[i] += investedSimilarity[INVESTING][organization]

    # influence 
    
    if organization in totalAssetDf.index:
        influences[i] += totalAssetDf.loc[organization]
    else:
        influences[i] += totalAssetDf.mean()
    if organization in serviceExpendDf.index:  
        influences[i] += serviceExpendDf.loc[organization]
    else:
        influences[i] += serviceExpendDf.mean()
    if organization in scraperDf.index:
        influences[i] += scraperDf.loc[organization]
    else:
        influences[i] += scraperDf.mean()
    
    # execution efficiency
    if organization in operationDf.index:
        executions[i] += operationDf.loc[organization]['total'] + operationDf.loc[organization]['service']
    else:
        executions[i] += operationDf['total'].mean() + operationDf['service'].mean()
    if organization in manforceDf.index:
        executions[i] += manforceDf.loc[organization]['man_force']
    else:
        executions[i] += manforceDf.mean()
    if organization in leaderDf.index:
        executions[i] += leaderDf.loc[organization]['leader_num']
    else:
        executions[i] += leaderDf.mean()
    if organization in servicePortionDf:
        executions[i] += executions[j].loc[organization]['service_portion']
    
    
similarities -= np.min(similarities)
similarities /= np.max(similarities)

influences -= np.min(influences)
influences /= np.max(influences)

executions -= np.min(executions)
executions /= np.max(executions)

similarityDict = dict(zip(all_organizations_except_investing, similarities))
influenceDict = dict(zip(all_organizations_except_investing, influences))
executionDict = dict(zip(all_organizations_except_investing, executions))
finalScoreDf= pd.DataFrame(columns=all_organizations_except_investing, index=[INVESTING])
for organization in all_organizations_except_investing:
    # similarity
    similarity = similarityDict[organization]
    influence = influenceDict[organization]
    execution = executionDict[organization]
    
    finalScoreDf.iloc[0][organization]=similarity* SIMILARITY_WEIGHT + influence* INFLUENCE_WEIGHT + execution * EXECUTION_WEIGHT

finalScoreDf = finalScoreDf.transpose().rename(columns={INVESTING:'score'})
finalScoreDf
    
    

3820 3821


,score
杭州市生态文化协会,10.728
帝斯曼维生素（上海）有限公司,10.728
浙江省人民医院,10.728
北京市华远集团公司,10.728
北京富润德斯文化发展有限公司,10.728
北京修实公益基金会,10.728
北京世纪银通投资有限公司,10.728
北京欧亚美展览服务有限公司,10.728
中国科学院上海微系统与信息技术研究所,10.728
云南商学院,10.728


In [197]:
finalScoreDf.sort_values(['score'] ,ascending=False)

,score
清华大学教育基金会,51.7559
广东省东风日产阳光关爱基金会,50.0824
中国医药卫生事业发展基金会,49.956
安徽金牛公益基金会,49.5384
河仁慈善基金会,48.8386
北京大学教育基金会,46.9552
广东省同心圆慈善基金会,46.2784
中国扶贫基金会,45.3461
北京中金公益基金会,42.5117
深圳市博时慈善基金会,42.1454
